# Monochrome Static Site Generator

## About

A custom-built `Python` Static Site-Generator used to generate pages for my personal [website](https://www.abhe.in/) and [blog](https://www.abhe.in/blog/).

Inspired by a tutorial by [Jahongir Rahmonov](https://rahmonov.me/posts/static-site-generator/).

## Usage

The script as it is, is used to fulfill the "back-end" needs of my website hosted here on GutHub Pages. However, it can easily be modified for use in different scenarios.

The following instructions are meant for Windows. Commands may vary for terminals other than `Powershell`.


### Clone Repository

* Go to project root and clone this repository.
  * If you have GitHub CLI installed, use the following commands: 
      ```powershell
    > cd <project-root> & mkdir <content>
    > gh repo clone #repolocation
    ```


### Virtual Envitonment Setup (Windows Powershell)

Instructions based on `virtualenv` [documentation](https://docs.python-guide.org/dev/virtualenvs/#lower-level-virtualenv).

* Install `virtualenv`

    ```powershell
    pip install virtualev
    # test installation
    virtualenv --version
    ```
* Go to project root and make a sub-directory for the content
    ```powershell
    > cd <project-root> & mkdir <content>
    ```
    * Remain in the project root.
* Initialize virtual environment
    ```powershell
    virtualenv venv
    ```
    * This will create a subdirectory 'venv' in the project root.
    * 'venv' is a general convention used globally and is readily available in ignore files. Feel free to change it, however, since sharing the folder generated by `virtualenv` is discouraged.
* Activate virtual environment
    ```powershell
    .\venv\Scripts\activate
    ```    
  * If that commond returns the following error:
      ```powershell
      ionic : <project-root>\venv\Scripts\activate.ps1 cannot be loaded 
      because running scripts is disabled on this system.
      
      For more information, see about_Execution_Policies 
      at https:/go.microsoft.com/fwlink/?LinkID=135170.
      
      At line:1 char:1
      + ~~~~~ 
          + CategoryInfo          : SecurityError: (:) [], PSSecurityException
          + FullyQualifiedErrorId : UnauthorizedAccess
      ```
  * Use the following command in the a separate `Powershell` terminal running as administrator before running the activation command again:
      ```powershell
      Set-ExecutionPolicy -ExecutionPolicy RemoteSigned
      ```
* You will know if the environment was activated successfully when `PS` in the terminal turns to `(venv)` 
* Install the `Pyhton` dependencies for this project
    ```powershell
    pip install -r requirements.txt
    ```
    * If you install additional libraries, you can update the `requirements.txt` file:
        ```powershell
        pip freeze > requirements.txt
        ```

## Python Libraries used:

* `os`
* `datetime`
* `markdown2`
* `jinja2`

## Script

Importing Libraries

In [25]:
import os
from datetime import datetime
from jinja2 import Environment, PackageLoader
from markdown2 import markdown

Generating Blog HTML from Markdown

In [26]:
with open('content/Post 0.md', 'r') as file:
    parsed_md = markdown(file.read(), extras=['metadata'])

    env = Environment(loader=PackageLoader('monochrome', 'templates'))
    post_detail_template = env.get_template('post-detail.html')

    data = {
        'content': parsed_md,
        'title': parsed_md.metadata['title'],
        'date': parsed_md.metadata['date']
    }

    post_detail_template.render(post=data)

*Might have to create an empty file named `monochrome.py` in the root directory for the above to work.*

In [27]:
POSTS = {}

for markdown_post in os.listdir('content'):
    file_path = os.path.join('content', markdown_post)

    with open(file_path, 'r') as file:
        POSTS[markdown_post] = markdown(file.read(), extras=['metadata'])

In [28]:
POSTS = {
    post: POSTS[post] for post in sorted(
        POSTS, key=lambda post: datetime.strptime(
            POSTS[post].metadata['date'], '%Y-%m-%d'),
            reverse=True)
}

In [29]:
env = Environment(loader=PackageLoader('monochrome', 'templates'))
index_template = env.get_template('index.html')
post_detail_template = env.get_template('post-detail.html')

In [30]:
index_posts_metadata = [
    POSTS[post].metadata for post in POSTS]

index_html_content = index_template.render(posts=index_posts_metadata)
print(index_html_content)

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>List of Posts</title>
</head>
<body>

<h1>My Blog Posts</h1>


  <p>
      <h2>1: <a href="posts/turkish-pide/">Turkish Cheese Pide</a> <small>2019-04-14</small></h2>
      Traditional Turkish pide stuffed with cheese. Great as an appetizer or for breakfast or lunch.
  </p>

  <p>
      <h2>2: <a href="posts/turkish-pide/">Turkish Cheese Pide</a> <small>2019-04-14</small></h2>
      Traditional Turkish pide stuffed with cheese. Great as an appetizer or for breakfast or lunch.
  </p>

  <p>
      <h2>3: <a href="posts/turkish-pide/">Turkish Cheese Pide</a> <small>2019-04-14</small></h2>
      Traditional Turkish pide stuffed with cheese. Great as an appetizer or for breakfast or lunch.
  </p>

  <p>
      <h2>4: <a href="posts/turkish-pide/">Turkish Cheese Pide</a> <small>2019-04-14</small></h2>
      Traditional Turkish pide stuffed with cheese. Great as an appetizer or for breakfast or lunch.
  </p>

  <p>
     

In [31]:
with open('output/index.html', 'w') as file:
    file.write(index_html_content)